In [41]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw
from ngsolve.krylovspace import CGSolver
from ttictoc import tic, toc

mesh = ngs.Mesh('whatever.vol')
# mesh.Curve(2)
# mesh.Refine()

In [42]:
%run J.ipynb

using 3d mesh with ne= 179611  elements and nv= 31186  points
Solving...
Number of iterations = 168/1000 | Residual = 8.85866142608333e-06
999.9999999999958
Solving...
Number of iterations = 326/1000 | Residual = 9.820525066381727e-06
999.9999999448601


In [43]:
from bhdata import BHCurves
fun_dw  = BHCurves(4)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil|ambient|default"
# nonlinear = "r_steel|l_steel|mid_steel"
nonlinear = "mid_steel"

lin = 0

BH data from TEAM 13 problem
returning energy


In [44]:
# print(HCurl.ndof)
# print(mesh.GetMaterials())

p = 1

HCurl = ngs.HCurl(mesh, order = p, nograds = True, dirichlet = 'ambient_face')
# HCurl = ngs.HCurl(mesh, order = p, nograds = True)
u,v = HCurl.TnT()


# Nonlinear:

maxit = 10000
tol2 = 1e-8
regb = 1e-12

A = ngs.GridFunction(HCurl)
B = ngs.curl(A)
normB = ngs.sqrt(B*B + regb)


if lin == 1: cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: nu0/2*B*B}, default = nu0/2*B*B).Compile()
else: cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: fun_w(normB)}, default = nu0/2*B*B).Compile()

def fun_W():
    with ngs.TaskManager(): res = ngs.Integrate(cf_energy - ngs.curl(Hs)*A, mesh)
    return res


if lin == 1: cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: nu0}, default = nu0).Compile()
else: cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: fun_dw(normB)/normB}, default = nu0).Compile()

rhs = ngs.LinearForm((cf_rhs*B*ngs.curl(v) - ngs.curl(Hs)*v)*ngs.dx)

def fun_dW(): #implicitly depending on A!
    with ngs.TaskManager(): rhs.Assemble()
    return rhs


Id = ngs.CF((1,0,0,
             0,1,0,
             0,0,1), dims=(3,3))

BBt = ngs.CF((B[0]*B[0], B[0]*B[1], B[0]*B[2],
              B[1]*B[0], B[1]*B[1], B[1]*B[2],
              B[2]*B[0], B[2]*B[1], B[2]*B[2]), dims = (3,3))


fun1 = fun_dw(normB)/normB
fun2 = (fun_ddw(normB) - fun_dw(normB)/normB)/(normB*normB)

if lin == 1: cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: nu0*Id}, default = nu0*Id).Compile()
else: cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: fun1*Id + fun2*BBt}, default = nu0*Id).Compile()

K_iter = ngs.BilinearForm(HCurl)
K_iter += (cf_iter*ngs.curl(u)*ngs.curl(v))*ngs.dx
C_iter = ngs.Preconditioner(K_iter, type = "local")

def fun_ddW():
    with ngs.TaskManager(): K_iter.Assemble()
    return K_iter


In [45]:
print("Using 3D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and " ,HCurl.ndof, "DOFs.\n ")

with ngs.TaskManager(): A.Set(ngs.CF((0,0,0)))

du = ngs.GridFunction(HCurl)
uo = ngs.GridFunction(HCurl)
wo = 1e12

for it in range(1,maxit+1):
    tic()
    # with ngs.TaskManager():
    #     K_iter.Assemble()
    #     rhs.Assemble()
    #     res = ngs.Integrate(cf_energy - Hs*ngs.curl(A), mesh)
    
    w  = fun_W()
    dw = fun_dW()
    da = fun_ddW()
    tm1 = toc()
    
    tic()
    # du.vec.data = da.mat.Inverse(HCurl.FreeDofs(), inverse="sparsecholesky") * dw.vec 
    # iterativeSolver = CGSolver(K_iter.mat, freedofs = HCurl.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
    with ngs.TaskManager():
        iterativeSolver = CGSolver(K_iter.mat, pre = C_iter.mat, tol  = 1e-2,  maxiter = maxit)
        du.vec.data = iterativeSolver * dw.vec 
    
    if len(iterativeSolver.residuals) == maxit: print("... Failure!")
    # print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    tm2 = toc()

    nrm = ngs.InnerProduct(du.vec,dw.vec)
    
    if it == 1:
        nrm0 = nrm
    
    # wn = 1e12
    if abs(wo-w) < tol2:
    # if abs(wn-w) < tol2:
    # if nrm/nrm0 < tol2:
        print("converged to desired tolerance")
        break
    # elif abs(wo-w) < tol2*1e-2:
    #     print("stopped early due to stagnation")
    #     break
    else:
        # linesearch
        uo.vec.data = A.vec.data
        wo = w
        alpha = 1
        for init in range(1,21):
            A.vec.data -= alpha*du.vec.data
            wn = fun_W()
            if wn < w - alpha*0.1*nrm:
                print("Iter: %2d | assem : %.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm1,tm2,iterativeSolver.iterations,alpha,w,nrm/nrm0))
                break
            else:
                alpha = alpha/2
                A.vec.data = uo.vec.data

Using 3D mesh with ne= 179611 elements and nv= 31186 points and  211010 DOFs.
 
Iter:  1 | assem : 1.21s | CG took 1.05s with   82 iterations | alpha : 1.00 | energy = 0.0000000000 | relres = 1.00e+00 |
Iter:  2 | assem : 1.46s | CG took 1.09s with  110 iterations | alpha : 1.00 | energy = -0.0802485230 | relres = 3.72e-05 |
Iter:  3 | assem : 1.56s | CG took 0.29s with   95 iterations | alpha : 1.00 | energy = -0.0802515063 | relres = 5.34e-09 |
converged to desired tolerance


In [49]:
L2 = ngs.L2(mesh, dim=3, order = 3)
BB = ngs.GridFunction(L2)

with ngs.TaskManager(): BB.Set(B)
BBnp = BB.vec.FV().NumPy()
# HCurl.ndof

Avec = A.vec.FV().NumPy()


import matplotlib.pyplot as plt
import numpy as np

# print(max(abs(BBnp)))
# print(BBnp.size,mesh.ne)

# plt.plot((BBnp))

In [47]:
# Draw(B, mesh, clipping={"z":-1, "dist":0.064})

In [48]:
ngs.Integrate(ngs.Norm(B),mesh)

6.904194388070618e-05